In [2]:
import pandas as pd
import numpy as np

In [3]:
# Read the data file and print the head of the data
df = pd.read_csv('XYZ/DataWithClusterLabel.csv')
# df = pd.read_csv('XYZ/Final_data.csv')
df.tail()

,id,total_impressions,total_clicks,total_apply_starts,actual_title,job_salary,cluster,job_state_freq,job_city_freq,advertiser_name_freq,employee_count_encoded,log_total_apply_starts,sqrt_total_apply_starts,boxcox_total_apply_starts,yeojohnson_total_apply_starts,click_imp_ratio
3423260,28526427,283333,18682,3742,AI Content Writer,45000.0,0,0.072537,0.004428,3.067247e-05,6.0,8.227643,61.171889,1.963695,2.127554,0.065937
3423261,28526750,340575,21237,4118,AI Content Writer,45000.0,0,0.026569,0.005121,3.067247e-05,6.0,8.323366,64.171645,1.965210,2.129968,0.062356
3423262,38767878,418525,26086,8310,Customer Care Representative,24000.0,3,0.010666,0.000099,5.842376e-07,2.0,9.025335,91.159201,1.974356,2.144533,0.062328
3423263,28526848,455158,28551,4818,AI Content Writer,45000.0,0,0.081970,0.008609,3.067247e-05,6.0,8.480322,69.411815,1.967543,2.133682,0.062728
3423264,38273190,657552,40531,4127,Online ELA Tutor,44000.0,0,0.012086,0.000478,2.921188e-07,2.0,8.325548,64.241731,1.965244,2.130021,0.061639


In [ ]:
df.columns

In [ ]:
df['total_apply_starts'].describe()

In [ ]:
# Print the count of missing values in the data
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
# Drop the rows with missing values in the columns 'job_state', 'job_city' and 'advertiser_name'
df = df.dropna(subset=['job_state', 'job_city','advertiser_name'])
df.shape

In [ ]:
3 # Print the count of missing values in the data
df.isnull().sum()

In [ ]:
# Print the unique values in the column 'job_state'
df['cluster'].value_counts()

In [ ]:
# Define a function to map the clusters to new clusters 
def map_cluster(cluster):
    if cluster in [1, 3, 5, 8, 9]:
        return 6
    elif cluster == 0:
        return 0
    elif cluster == 2:
        return 1
    elif cluster == 4:
        return 2
    elif cluster == 6:
        return 3
    elif cluster == 7:
        return 4
    else:
        return 5

# Apply the function to create a new column
df['new_cluster'] = df['cluster'].apply(map_cluster)

In [ ]:
# Print the unique values in the column 'new_cluster'
df['new_cluster'].value_counts()

In [ ]:
# Fill missing job salaries with the mean salary within each job_state and new_cluster group
df['job_salary'] = df.groupby(['job_state', 'new_cluster'])['job_salary'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
3 # Print the count of missing values in the data
df.isnull().sum()

In [ ]:
# Fill missing values in the 'employee_count' column with 'Unknown'
df['employee_count'] = df['employee_count'].fillna('Unknown')


In [ ]:
3 # Print the count of missing values in the data
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
# Create a copy of the data
df_og = df.copy()

In [ ]:
# Print the unique number of values of 'advertiser_name'
df['advertiser_name'].nunique()

In [ ]:
#                                     FREQUENCY ENCODING


# Define a function to perform frequency encoding
def frequency_encoding(column):
    freq = df[column].value_counts() / len(df)
    return df[column].map(freq)

df['job_state_freq'] = frequency_encoding('job_state')
df['job_city_freq'] = frequency_encoding('job_city')
df['advertiser_name_freq'] = frequency_encoding('advertiser_name')

In [ ]:
# #                                     TARGET ENCODING

# categorical_columns = ['job_state', 'job_city', 'advertiser_name']
# target_column = 'total_impressions'

# # Function to perform target encoding
# def target_encode(df, categorical_columns, target_column):
#     encoded_df = df.copy()
#     for col in categorical_columns:
#         # Calculate the mean of the target column for each category
#         target_mean = df.groupby(col)[target_column].mean()
#         # Map the mean values to the original dataframe
#         encoded_df[col + '_encoded'] = df[col].map(target_mean)
#     return encoded_df

# # Perform target encoding
# encoded_df = target_encode(df, categorical_columns, target_column)

# # Display the encoded dataframe
# encoded_df.head()

In [ ]:
df.head()

In [ ]:
 # Drop the columns 'job_state', 'job_city' and 'advertiser_name'
df = df.drop(['job_state', 'job_city', 'advertiser_name'], axis=1)

In [ ]:
df.head()

In [ ]:
# Print value_counts of 'employee_count'
df['employee_count'].value_counts()

In [ ]:
# Import the OrdinalEncoder class from sklearn
from sklearn.preprocessing import OrdinalEncoder

# Define the order of the categories
employee_count_order = ['1-49', '50-149', '150-249', '250-499', '500-749', '750-999' ,'1000+', 'Unknown']

# Create an instance of OrdinalEncoder with the specified order
ordinal_encoder = OrdinalEncoder(categories=[employee_count_order])

# Encode the 'employee_count' column
df['employee_count_encoded'] = ordinal_encoder.fit_transform(df[['employee_count']])
df = df.drop('employee_count', axis=1)

In [ ]:
df.head()

In [ ]:
from scipy import stats
from sklearn.preprocessing import PowerTransformer

# Applying various transformations to the 'total_apply_starts' column
df['log_total_apply_starts'] = np.log1p(df['total_apply_starts'])  # Using log1p to handle zeros
df['sqrt_total_apply_starts'] = np.sqrt(df['total_apply_starts'])
df['boxcox_total_apply_starts'], _ = stats.boxcox(df['total_apply_starts'] + 1)  # Adding 1 to handle zeros

# Using Yeo-Johnson transformation which can handle zeros and negative values
pt = PowerTransformer(method='yeo-johnson')
df['yeojohnson_total_apply_starts'] = pt.fit_transform(df[['total_apply_starts']])

# Display summary statistics for the original and transformed columns
df[['total_apply_starts', 'log_total_apply_starts', 'sqrt_total_apply_starts', 'boxcox_total_apply_starts', 'yeojohnson_total_apply_starts']].describe()


In [ ]:
# Create a new column 'click_imp_ratio' by dividing 'total_clicks' by 'total_impressions'
df['click_imp_ratio'] = df['total_clicks'] / df['total_impressions']

In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

# Calculate the correlation matrix
correlation_matrix = df.drop(['id','total_impressions','total_clicks','actual_title','sqrt_total_apply_starts','boxcox_total_apply_starts','yeojohnson_total_apply_starts'],axis=1).corr()

# Plot the correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

In [3]:
# Enumerate the columns of the dataframe
for i,j in enumerate(df.columns):
    print(i,j)

0 id
1 total_impressions
2 total_clicks
3 total_apply_starts
4 actual_title
5 job_salary
6 cluster
7 job_state_freq
8 job_city_freq
9 advertiser_name_freq
10 employee_count_encoded
11 log_total_apply_starts
12 sqrt_total_apply_starts
13 boxcox_total_apply_starts
14 yeojohnson_total_apply_starts
15 click_imp_ratio


In [159]:
from sklearn.model_selection import train_test_split

# Select features and target variable for the model
X = df1.iloc[:, np.r_[5,6,7,8,9,10]]
y = df1['log_total_apply_starts']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1028)


In [160]:
# Print the shapes of the training and testing sets
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2666253, 6)
(666564, 6)
(2666253,)
(666564,)


In [157]:
X_train.head()

,job_salary,cluster,job_state_freq,job_city_freq,advertiser_name_freq,employee_count_encoded
1048606,62319.414712,7,0.029213,0.003268,8.763563e-07,0.0
838012,91750.000000,2,0.038558,0.001720,2.670842e-03,3.0
154038,78584.651070,9,0.026569,0.005121,2.728389e-03,7.0
168622,70720.000000,8,0.034400,0.000010,1.089188e-01,1.0
134033,52000.000000,8,0.081970,0.001761,1.089188e-01,1.0


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

# Function to train and evaluate multiple regression models
def train_and_evaluate_models(X_train, y_train, X_test, y_test):
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(),
        'Decision Tree': DecisionTreeRegressor()
    }

    # Train and evaluate each model
    for name, model in models.items():
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        train_r2 = r2_score(y_train, y_train_pred)
        train_mse = mean_squared_error(y_train, y_train_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        test_mse = mean_squared_error(y_test, y_test_pred)
     
        print(f'{name} Model:')
        print(f'Training R²: {train_r2:.4f}')
        print(f'Training MSE: {train_mse:.4f}')
        print(f'Testing R²: {test_r2:.4f}')
        print(f'Testing MSE: {test_mse:.4f}')
        print('-' * 30)

# Train and evaluate models using the training and testing sets
train_and_evaluate_models(X_train, y_train, X_test, y_test)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Function to train and evaluate multiple advanced regression models
def train_and_evaluate_models(X_train, y_train, X_test, y_test):
    models = {
        'Gradient Boosting': GradientBoostingRegressor(),
        'AdaBoost': AdaBoostRegressor(),
        'XGBoost': XGBRegressor(),
        'LightGBM': LGBMRegressor()
    }

    # Train and evaluate each model
    for name, model in models.items():
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        train_r2 = r2_score(y_train, y_train_pred)
        train_mse = mean_squared_error(y_train, y_train_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        test_mse = mean_squared_error(y_test, y_test_pred)
     
        print(f'{name} Model:')
        print(f'Training R²: {train_r2:.4f}')
        print(f'Training MSE: {train_mse:.4f}')
        print(f'Testing R²: {test_r2:.4f}')
        print(f'Testing MSE: {test_mse:.4f}')
        print('-' * 30)

# Train and evaluate models using the training and testing sets
train_and_evaluate_models(X_train, y_train, X_test, y_test)


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score, mean_squared_error

def train_and_evaluate_models(X_train, y_train, X_test, y_test):
    models = {
        'Gradient Boosting': GradientBoostingRegressor(),
        'AdaBoost': AdaBoostRegressor(),
        'XGBoost': XGBRegressor(),
        'LightGBM': LGBMRegressor()
    }

    feature_names = X_train.columns if hasattr(X_train, 'columns') else [f'Feature {i}' for i in range(X_train.shape[1])]

    for name, model in models.items():
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        train_r2 = r2_score(y_train, y_train_pred)
        train_mse = mean_squared_error(y_train, y_train_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        test_mse = mean_squared_error(y_test, y_test_pred)
        
        print(f'{name} Model:')
        print(f'Training R²: {train_r2:.4f}')
        print(f'Training MSE: {train_mse:.4f}')
        print(f'Testing R²: {test_r2:.4f}')
        print(f'Testing MSE: {test_mse:.4f}')
        print('\n')

        # Print sorted feature importance if available
        if hasattr(model, 'feature_importances_'):
            importance = model.feature_importances_
            sorted_indices = importance.argsort()[::-1]  # Get indices of features sorted by importance
            print('Sorted Feature Importance:')
            for idx in sorted_indices:
                print(f'{feature_names[idx]}: {importance[idx]:.4f}')
        
        print('-' * 30)

# Example usage
# Assuming you have your data split into X_train, y_train, X_test, y_test
train_and_evaluate_models(X_train, y_train, X_test, y_test)


In [161]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

def train_and_evaluate_glm(X_train, y_train, X_test, y_test):
    # Adding a constant term for the intercept
    X_train_const = sm.add_constant(X_train)
    X_test_const = sm.add_constant(X_test)
    
    # Fit the Poisson regression model
    poisson_model = sm.GLM(y_train, X_train_const, family=sm.families.Poisson()).fit()

    # Make predictions
    y_train_pred = poisson_model.predict(X_train_const)
    y_test_pred = poisson_model.predict(X_test_const)

    # Calculate metrics
    train_r2 = r2_score(y_train, y_train_pred)
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)

    # Print the results
    print('Poisson GLM Model:')
    print(f'Training R²: {train_r2:.4f}')
    print(f'Training MSE: {train_mse:.4f}')
    print(f'Testing R²: {test_r2:.4f}')
    print(f'Testing MSE: {test_mse:.4f}')
    print('-' * 30)

# Example usage:
# Assuming you have your data split into train and test sets
# X_train, X_test, y_train, y_test

# Call the function with your data
train_and_evaluate_glm(X_train, y_train, X_test, y_test)


Poisson GLM Model:
Training R²: 0.2451
Training MSE: 1.1584
Testing R²: 0.2455
Testing MSE: 1.1570
------------------------------


In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import pandas as pd

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Initialize the XGBoost regressor
xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror')

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_regressor, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Predictions on training and testing data
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Calculate metrics
def calculate_metrics(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return r2, mse, rmse

train_r2, train_mse, train_rmse = calculate_metrics(y_train, y_train_pred)
test_r2, test_mse, test_rmse = calculate_metrics(y_test, y_test_pred)

# Print metrics
print(f"Training Metrics: R2: {train_r2:.4f}, MSE: {train_mse:.4f}, RMSE: {train_rmse:.4f}")
print(f"Testing Metrics: R2: {test_r2:.4f}, MSE: {test_mse:.4f}, RMSE: {test_rmse:.4f}")

# Print best parameters
print("Best Model Parameters:", grid_search.best_params_)

# Feature importance
feature_importances = pd.DataFrame({'Feature': X_train.columns, 'Importance': best_model.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print("Sorted Feature Importances:\n", feature_importances)

# More Fine tuning

In [59]:
from sklearn.model_selection import train_test_split

X = df1.iloc[:,np.r_[5,6,7,8,9,10]]
y = df1['log_total_apply_starts']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1028)

In [149]:
# Define the XGBoost regressor model
xgb_regressor = xgb.XGBRegressor(max_depth = 10, random_state = 1028, n_estimators = 100, alpha = 0.1, eval_metric='rmse', objective='reg:squarederror', )

# # Fit the model on the training data
xgb_regressor.fit(X_train, y_train)

XGBRegressor(alpha=0.1, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [150]:
y_train_pred = xgb_regressor.predict(X_train)
y_test_pred = xgb_regressor.predict(X_test)

In [151]:
# # Calculate metrics
train_r2 = r2_score(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)

test_r2 = r2_score(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)

In [152]:
# # Print metrics
print(f"Training Metrics: R2: {train_r2:.4f}, MSE: {train_mse:.4f}, RMSE: {train_rmse:.4f}")
print(f"Testing Metrics: R2: {test_r2:.4f}, MSE: {test_mse:.4f}, RMSE: {test_rmse:.4f}")
print('-' * 30)

feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': xgb_regressor.feature_importances_})
sorted_feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
print(sorted_feature_importance)

Training Metrics: R2: 0.5441, MSE: 0.6997, RMSE: 0.8365
Testing Metrics: R2: 0.5082, MSE: 0.7541, RMSE: 0.8684
------------------------------
                  Feature  Importance
4    advertiser_name_freq    0.544168
0              job_salary    0.150411
1                 cluster    0.121508
5  employee_count_encoded    0.081857
2          job_state_freq    0.060012
3           job_city_freq    0.042044
